<center> <img src="https://impassivewealth.com/wp-content/uploads/2020/10/ActivePassive-768x241.png" align="center" width="960" height="480"/> </center>

<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> I.F. Blas César Sánchez Barba </font> <br> <br>
   
<center> <font color= #555555> <font size = 4> Jun 2021 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/if723286/Portafolio-Activo-Pasivo'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Lab 1 </font> <br> <br> <font color= #555555> <font size = 5> Active Investment vs Pasive Investment. Which one is better?  </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  Include here a description of the whole work, use no more than 400 words, you can use **Bold** and *Italic* highlights, also you can inclue [links](www.iteso.mx). Usually is a good strategy to generate this text at the very end of the process, since you have to include all aspects of the work, from a brief introduction, to problem definitions and methods used, to finally the results found. </font>

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

<hr style="border:0.02in solid gray"> </hr>

This is an introduction, normally, it is done at the very end, when all the other content in the notebook is completed but just before the abstract (which is short version of the introduction) ... 

<br>

# <font color= #6B6B6B> 2. Install/Load Packages and Depedencies </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

Finally, from the notebook it will be possible to acces other files within the mounted folder

### <font color= #6B6B6B> 2.1 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas>=1.1.1 
- numpy>=1.19.1
- jupyter>=1.0.0
- _Add any other here_

In [ ]:
# Install all the pip packages in the requirements.txt
%%capture
import sys
!{sys.executable} -m pip install -r requirements.txt

### <font color= #6B6B6B> 2.2 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

- files/precios.csv : Precios históricos
- files/datos.txt : Otros datos históricos

<br>

# <font color= #6B6B6B> 3. Data Description </font>

<hr style="border:0.02in solid gray"> </hr>

Here we explain the data

### <font color= #6B6B6B> 3.1 Selection of stocks for portfolio </font> <font color= #555555>

We want to create our own portfolio. In order to do that is necesarry ti know which stocks are the most important ones in the NAFTRAC.

Since NAFRAC is a ETF that replies the S&P IPC 35, and the S&P IPC includes only the most strong enterprices in the Mexican market based in its level of capitalization, we need to obtain those stock that always appear in the NAFTRAC

First of all, lets get all the stocks that appered at lest once during the last two years

In [ ]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

In [ ]:
# directory containing the csv files
directory = 'files'



columns = ['Ticker']
df_global = pd.DataFrame(columns=columns)

# loop through all the files in the directory
for filename in os.listdir(directory):
    # check if the file is a csv file
    if filename.endswith(".csv"):
        # read the csv file and store it in a data frame
        df = pd.read_csv(os.path.join(directory, filename), skiprows=2, nrows=37 , usecols=["Ticker", 'Peso (%)', 'Precio'])
        # append the data frame to the list
        df_global = pd.concat([df_global, df])      

Now we want to obtain how many times each stock appear in the NAFTRAC

In [ ]:
df_global['Ticker'] = df_global['Ticker'].map(lambda x: x.replace('*',''))
df_global['Ticker'] = df_global['Ticker'].map(lambda x: x.replace('.',''))
df_global['Ticker'] = df_global['Ticker'].apply(lambda x: "{}{}".format(x, '.MX'))

Then lets just select those stocks that appear all and every single time in the NAFTRAC

In [ ]:
df_global_list = df_global['Ticker'].value_counts(sort=True,ascending=False).index[:34].tolist()

There were some errors, so lets fiz them before going into the portfolios

In [ ]:
df_global_list.remove('LIVEPOLC1.MX')
df_global_list.remove('MXN.MX')
df_global_list.append('LIVEPOLC-1.MX')

In [ ]:
p_portfolio = df_global.head(33)
p_portfolio['Precio'] = p_portfolio['Precio'].astype(str)
p_portfolio['Precio'] = p_portfolio['Precio'].map(lambda x: x.replace(',',''))
p_portfolio['Precio'] = p_portfolio['Precio'].astype(float)
p_portfolio['Peso (%)'] = p_portfolio['Peso (%)'].apply(lambda x: x/100)

# <font color= #6B6B6B> 4. Pasive investment  </font>

<hr style="border:0.02in solid gray"> </hr>

In [ ]:
p_portfolio = p_portfolio.sort_values('Ticker')
p_portfolio = p_portfolio.reset_index(drop=True)

Lets get the value to invest in each and every stock

In [ ]:
p_portfolio['Value'] = p_portfolio['Peso (%)'] * 1000000
january_2021 = p_portfolio.Value.sum()

Lets call the prices of the stocks included in our portfolio

In [ ]:
tickers = df_global_list

data = yf.download(tickers = tickers, start= '2021-01-01', end='2023-02-01', interval = "1mo")

In [ ]:
data = data['Adj Close']
data = data.transpose()

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.insert(0, 'Peso (%)', p_portfolio['Peso (%)'])
data.insert(0, 'Ticker', p_portfolio['Ticker']) 

In [ ]:
data = data.dropna()

In [ ]:
primero = pd.DataFrame({'capital': [january_2021]})

In [ ]:
results=[]
for j in range (0,24):
    suma = 0
    for i in range (0, 31):
        X = 0
        X = (data.iloc[i,j+3]/data.iloc[i,j+2]-1)*data.iloc[i,1]
        if X>0:
            suma += X
    results.append(suma)

In [ ]:
df = pd.DataFrame (results, columns = ['rendimiento'])
df.loc[-1] = [0]  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 

In [ ]:
import datetime
dates=[] 
# range of dates
date_range = pd.period_range(
    start='2021-01-01', periods=26, freq='M')
  
# timestamp range
timestamp_range = [x.to_timestamp() for x in date_range]
  
# iterating through timestamp range
for i in timestamp_range:
    dates.append(i)

df_pasiva = pd.DataFrame (dates, columns = ['timestamp'])

In [ ]:
df_pasiva.insert(1, 'capital', primero['capital'])
df_pasiva.insert(2, 'rendimiento', df['rendimiento'])

In [ ]:
df_pasiva.iloc[1,1] = df_pasiva.iloc[0,1] #la primera es la fila y la segunda es la columna

for i in range (0,23):
    df_pasiva.iloc[i+2,1] = df_pasiva.iloc[i+1,1]*(1 + df_pasiva.iloc[i+1,2])

In [ ]:
acumulado=[]
for i in range(0,23):
    x  = df_pasiva.iloc[i+1,1]/df_pasiva.iloc[0,1]-1
    acumulado.append(x)
acumulado = pd.DataFrame (acumulado, columns = ['rendimiento acumulado'])
    

In [ ]:
df_pasiva.insert(3, 'rendimiento acumulado', acumulado['rendimiento acumulado'])
df_pasiva = df_pasiva.dropna()

In [ ]:
df_pasiva



<br>

# <font color= #6B6B6B> 5. Active investment  </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

In [ ]:
data = yf.download(tickers = tickers, start= '2021-01-01', end='2023-02-01', interval = "1mo")
data = data['Adj Close']
ret1 = data.pct_change().dropna()
ret1.plot(figsize=(9,7),grid=True);

In [ ]:
# Rendimientos esperados, volatilidad, matriz de covarianza y matriz de correlacion
tabla1 = pd.DataFrame(data={'Media1':ret1.mean(),
                            'Volatilidad1':ret1.std()
                            }, index=ret1.columns)

cov1 = ret1.cov()
corr1 = ret1.corr()

In [ ]:
#Tasa libre de riesgo
rf1 = 0.11 / 12

In [ ]:
# Construcción de parámetros
S1 = np.diag(tabla1.iloc[:,1])
Sigma1 = S1.dot(corr1).dot(S1)

Eind1 = tabla1.iloc[:,0]

# Función objetivo
def varianza(w, Sigma1):
    return w.T.dot(Sigma1).dot(w)

# Dato inicial
n1 = len(Eind1)
w01 = np.ones(n1) / n1
# Cotas de las variables
bnds1 = ((0, 1),) * n1
# Restricciones
cons1 = {'type': 'eq', 'fun': lambda w: w.sum() - 1}

# Portafolio de mínima varianza
minimavar1 = minimize(fun=varianza,
                   x0=w01,
                   args=(Sigma1,),
                   bounds=bnds1,
                   constraints=cons1
                  )
minimavar1

In [ ]:
# Pesos, rendimiento, riesgo y Radio de Sharpe del portafolio de mínima varianza

w_minvar1 = minimavar1.x
E_minvar1 = Eind1.T.dot(w_minvar1)
s_minvar1 = varianza(w_minvar1, Sigma1)**0.5
RS_minvar1 = (E_minvar1 - rf1) / s_minvar1
w_minvar1, E_minvar1, s_minvar1, RS_minvar1

Verificamos que los pesos sean correctos

In [ ]:
w_minvar1.sum()

In [ ]:
data = data.transpose()

In [ ]:
data.insert(0, 'Peso (%)', w_minvar1)

In [ ]:
df_activa = pd.DataFrame (dates, columns = ['timestamp'])

In [ ]:
ret1=ret1.transpose()
ret1.insert(0, 'Peso (%)', w_minvar1)

In [ ]:
df = pd.DataFrame(ret1)
# Define the condition to filter the rows
condition = df['Peso (%)'] > 0.001

# Filter the DataFrame based on the condition
filtered_ret1 = df[condition]

In [ ]:
# Create a list of values to filter by
values_to_filter = filtered_ret1.index.values

filtered_data = data[data.index.isin(values_to_filter)]

Voy a invertir $900,000 al inicio para poder tener CASH y ser capaz de hacer rebalanceos ir haciendo rebalanceos mes a mes

In [ ]:
filtered_data.insert(1, 'Value', filtered_data['Peso (%)']*900000)

In [ ]:
filtered_data.insert(2, 'Stocks', filtered_data['Value']/filtered_data.iloc[:,3])

In [ ]:
stocks = filtered_data.iloc[:, [1, 2]]


Hay que obtener los valores de nuestro portafolio para el primer periodo

In [ ]:
df_activa.insert(1, 'capital',  filtered_data['Value'].sum())
df_activa.insert(2, 'rendimiento', 0)
df_activa.insert(3, 'rendimeinto acumulado', 0)
df_activa.insert(4, 'titulos totales', filtered_data['Stocks'].sum())
df_activa.insert(5, 'titulos_c_v', 0)
df_activa.insert(6, 'comision', filtered_data['Value'].sum()*0.004) #es la comision que cobra Kuspit
df_activa.insert(7, 'comision acumulada', filtered_data['Value'].sum()*0.04)

In [ ]:
for j in range(0,24):  
    for i in range(0,9):
        filtered_data.iloc[i,1] = filtered_data.iloc[i,1]*(1+filtered_ret1.iloc[i,j+1]) #Para actualizar cuanto vale la inversion del portafolio en base a moviemientos del mercado
        # Hasta esto momento solo se ve reflajado en filtered_data el rendimiento de mercado, falta sumarle lo que compre o vendi de acciones por que los cambios en el precio (rebalanceo)

        #Para los balanceos de acciones (Aqui cambiamos la ponderacion de cada activo en el portafolio)
        if filtered_ret1.iloc[i,j+1] < -0.05:  #Row,Column
            filtered_data.iloc[i,0]= filtered_data.iloc[i,0]-0.025

        elif filtered_ret1.iloc[i,j+1] > 0.05:  
            filtered_data.iloc[i,0]= filtered_data.iloc[i,0]+0.025
        else:
            filtered_data.iloc[i,0]= filtered_data.iloc[i,0]

        #Para las compras de acciones y con ello cabiar el valor de mi portafolio
        if filtered_ret1.iloc[i,j+1] < -0.05:  #Row,Column
            filtered_data.iloc[i,1] = filtered_data.iloc[i,1]*(1-0.025)

        elif filtered_ret1.iloc[i,j+1] > 0.05:  
            filtered_data.iloc[i,1] = filtered_data.iloc[i,1]*(1+0.025)
        else:
            filtered_data.iloc[i,1] = filtered_data.iloc[i,1]*(1-0.025)

        
        
        filtered_data.iloc[i,2] = filtered_data.iloc[i,1]/filtered_data.iloc[i,j+3] #Para numero de stocks 


    #Aqui vamos a poner todos los cambios de las variables que se hacen mes a mes por el rebalanceo    
    df_activa.iloc[j+1,1]=filtered_data.Value.sum()
    df_activa.iloc[j+1,2]=(filtered_ret1.iloc[:,j+1]*filtered_ret1.iloc[:,j]).sum()
    df_activa.iloc[j+1,3]=(1+df_activa.iloc[j+1,2])*(1+1+df_activa.iloc[j,2])-1
    df_activa.iloc[j+1,4]=filtered_data.Stocks.sum()
    df_activa.iloc[j+1,5]=df_activa.iloc[j+1,4]-df_activa.iloc[j,4]
    if  df_activa.iloc[j+1,5]>0:
         df_activa.iloc[j+1,6]=df_activa.iloc[j+1,5]*0.004 #La comison que cobra kuspit
    else:
        df_activa.iloc[j+1,6]=df_activa.iloc[j+1,5]*0.004*-1
    df_activa.iloc[j+1,7]=df_activa.iloc[j,7]+df_activa.iloc[j+1,6]
    
df_activa = df_activa.drop(df_activa.index[-1])

In [ ]:
df_activa

# <font color= #6B6B6B> <font size = 6> 5. Results </font>

<hr style="border:0.02in solid gray"> </hr>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# In here you present the general results, and also, a discussion of such results.

<br>

## <font color= #6B6B6B> 5.1 Results of first processes </font>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# Separate your results according to the main processes you are working on

<br>

## <font color= #6B6B6B> 5.2 Results of another proces </font>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# You can have several processes with which you are obtaining the results you need.

<br>

# <font color= #6B6B6B> <font size = 6> 6. References </font>

<hr style="border:0.02in solid gray"> </hr>

In [ ]:
# DELETE THIS CELL IN YOUR FILE

# Citations, and other references you are using.

# DO INCLUDE THE FOLLOWING CITATION FOR THIS TEMPLATE

[1] Munnoz, 2020. Python project template. https://github.com/iffranciscome/python-project. (2021).

<br>

# <font color= red> TESTS (DO NOT INCLUDE IN THE FINAL NOTEBOOK) </font>

<hr style="border:0.02in solid red"> </hr>

Next are a series of tests to verify that everything is working ok. From correctly installed and loaded packages, to import other scripts, print plots with plotly

In [ ]:
import pandas as pd
import data as dt

In [ ]:
# -- TEST 1 : 
# verify that the script is being read
print(dt.dict_test)

In [ ]:
# -- TEST 2 :
# verify that installed pandas module works correctly
df_dict_test = pd.DataFrame(dt.dict_test, index=[0,1])
print(df_dict_test)

In [ ]:
# -- TEST 3 :
# verify you can use plotly and visualize plots in jupyter notebook

from plotly.offline import iplot   # print the plot inside jupyter and offline
import plotly.graph_objects as go  # functions to create plots

# example data
df = pd.DataFrame({'column_a': [1, 2, 3, 4, 5], 'column_b': [1, 2, 3, 4, 5]})
# basic plotly plot
data = [go.Bar(x=df['column_a'], y=df['column_b'])]

# instruction to view it inside jupyter
iplot(data)

In [ ]:
# -- TEST 4 :
# verify you can use plotly and visualize plots in web browser locally

import plotly.io as pio            # to define input-output of plots
pio.renderers.default = "browser"  # to render the plot locally in your default web browser

# basic plotly plot
plot_data = go.Figure(go.Bar(x=df['column_a'], y=df['column_b']))

# instruction to view it in specified render (in this case browser)
plot_data.show()